In [1]:
import pandas as pd

In [13]:
# flow data
daily_data = pd.read_csv('./training_dataset/daily_dataset.csv')
hourly_data = pd.read_csv('./training_dataset/hourly_dataset.csv')
fivemin_data = pd.read_csv('./training_dataset/per5min_dataset.csv')
# other data
weather_data = pd.read_csv('./training_dataset/weather.csv')
epidemic_data = pd.read_csv('./training_dataset/epidemic.csv')
# 筛选训练数据
daily_train = daily_data[daily_data['train or test']=='train']
hourly_train = hourly_data[hourly_data['train or test']=='train']
fivemin_train = fivemin_data[fivemin_data['train or test']=='train']
weather_train = weather_data[weather_data['train or test']=='train']
epidemic_data = epidemic_data[epidemic_data['train or test']=='train']

In [24]:
len(hourly_train['time'].unique())

5064

In [21]:
fivemin_train

,time,flow_1,flow_2,flow_3,flow_4,flow_5,flow_6,flow_7,flow_8,flow_9,...,flow_12,flow_13,flow_14,flow_15,flow_16,flow_17,flow_18,flow_19,flow_20,train or test
0,2022-01-01 00:05:00,3.8,1.9,6.5,7.4,0.5,NaN,1.7,0.3,0.7,...,0.428,0.2,0.5,0.2,0.6,1.0,18.379,0.252,0.3,train
1,2022-01-01 00:10:00,3.7,1.7,6.5,7.5,0.7,7.3,1.9,0.2,0.7,...,0.356,0.3,0.5,0.3,0.7,0.8,NaN,0.232,0.4,train
2,2022-01-01 00:15:00,3.7,2.0,6.3,6.9,0.6,7.5,1.9,0.3,0.7,...,0.292,0.2,0.7,0.4,0.6,1.0,NaN,0.220,0.2,train
3,2022-01-01 00:20:00,3.3,2.5,6.3,6.5,0.5,7.0,1.7,0.2,0.5,...,0.340,0.2,0.7,0.3,0.5,1.0,NaN,0.258,0.4,train
4,2022-01-01 00:25:00,3.5,2.4,6.4,6.4,0.7,6.8,1.5,0.2,0.6,...,0.496,0.3,0.5,0.3,0.4,0.9,NaN,0.238,0.3,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66811,2022-08-20 23:40:00,3.3,2.4,8.7,7.9,0.4,8.6,2.0,0.3,1.0,...,NaN,0.5,0.5,0.3,0.8,1.1,NaN,NaN,0.5,train
66812,2022-08-20 23:45:00,3.0,2.5,9.0,7.6,0.4,7.7,2.3,0.2,0.9,...,NaN,0.4,0.5,0.4,0.7,1.2,NaN,NaN,0.3,train
66813,2022-08-20 23:50:00,15.7,2.2,8.6,7.3,0.5,7.7,1.9,0.3,0.7,...,NaN,0.6,0.5,0.4,0.8,1.1,NaN,NaN,0.3,train
66814,2022-08-20 23:55:00,14.8,2.3,10.0,8.5,0.6,8.6,2.0,0.2,0.8,...,NaN,0.5,0.4,0.4,0.8,1.1,NaN,NaN,0.3,train
